<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/AI_generated_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.6

# Text Classification

In [2]:
import pandas as pd
import numpy as np
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers

ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

df = pd.read_csv('https://raw.githubusercontent.com/spe301/Wikipedia-Capstone/main/Data/HumanText.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis='columns')
df2 = pd.read_csv('https://raw.githubusercontent.com/spe301/Wikipedia-Capstone/main/Data/Generated%20(1).csv').drop(['Unnamed: 0'], axis='columns')
blog_df = pd.concat([df.sample(n=180), df2])

df = pd.read_csv('https://raw.githubusercontent.com/spe301/dsc-phase-4-project/main/data/judge-1377884607_tweet_product_company.csv', engine='python',encoding='latin-1')
df = df.drop(6)
df = df.loc[df['is_there_an_emotion_directed_at_a_brand_or_product'] != "I can't tell"]
ix = list(df.index)
clean = []
for i in range(len(ix)):
  clean.append(df['tweet_text'][ix[i]].replace('.@', '').replace('.', '').replace('!', '').replace('@', '').replace('?', '').replace(':', '')
  .replace(',', '').replace(';', '').lower())

df['tweet'] = clean
tweet_df = df.drop(['tweet_text', 'emotion_in_tweet_is_directed_at'], axis='columns')

In [9]:
def Preprocess(df, data_str, target_str):
  '''fully preprocesses a dataframe of Text Documents for classification'''
  avg = 0
  for i in range(len(df[data_str])):
    avg += len(list(df[data_str])[i].split(' '))/len(df)
  pad = int(avg)
  text, labels = dl.ModelReadyText1(df[data_str], df[target_str], pad)
  return text, labels

blogs, blog_labels = Preprocess(blog_df, 'Text', 'AI')
tweets, tweet_labels = Preprocess(tweet_df, 'tweet', 'is_there_an_emotion_directed_at_a_brand_or_product')

In [11]:
import numpy as np

def GetVocab(text_series):
  words = []
  for i in range(len(text_series)):
    word_lst = list(text_series)[i].replace('\n', ' ').split(' ')
  for word in word_lst:
    words.append(word.replace('.', '').replace(',', '').replace(' ', '').replace('"', '').replace(':', '').replace(';', '').replace('!', ''))
  vocab = len(np.unique(words))
  return vocab

blogs_vocab = GetVocab(blog_df['Text'])
tweets_vocab = GetVocab(tweet_df['tweet'])

9

In [12]:
blogs_grid = {'output_dim': [len(np.unique(blog_labels))], 'embedding': } 

TypeError: ignored

# Image Classification

# Tabular Classification

# Tabular Regression